In [13]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pandas as pd
import gensim
from gensim import corpora
from nltk.corpus import stopwords
import spacy


data = pd.read_csv("data.csv")
print(data['content'].info())
dataFiltered = data["content"].drop_duplicates().dropna()[:35000]

stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
dataFiltered=dataFiltered.apply(remove_stopwords)

# nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

text_list=dataFiltered.tolist()
#print(text_list[5])
tokenized_reviews = lemmatization(text_list)
#print(tokenized_reviews[5])

dictionary = corpora.Dictionary(tokenized_reviews)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus=doc_term_matrix, id2word=dictionary, num_topics=20, random_state=100, chunksize=100, passes=10)
print("-"*10, "\n", "Topics\n", ldamodel.print_topics(num_words=10))



<class 'pandas.core.series.Series'>
RangeIndex: 50000 entries, 0 to 49999
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
50000 non-null  object
dtypes: object(1)
memory usage: 390.8+ KB
None


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)
vis


C:\Users\eslam\AppData\Roaming\Python\Python310\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.115531 -0.099317       1        1  23.840371
16    -0.171559 -0.062162       2        1   9.120863
13    -0.141226 -0.163279       3        1   6.770529
19    -0.129501 -0.044767       4        1   6.699484
14    -0.068732 -0.169848       5        1   5.593618
11    -0.087275 -0.063152       6        1   5.263973
5      0.044770  0.055927       7        1   5.223218
4      0.038678  0.117881       8        1   4.224477
0     -0.011697  0.186198       9        1   4.074095
17    -0.134896 -0.009329      10        1   4.014718
6     -0.083180  0.129542      11        1   3.867943
2      0.078670 -0.153168      12        1   3.717334
7     -0.053907  0.129825      13        1   3.492435
9     -0.054537  0.239435      14        1   3.381902
18    -0.025125 -0.058324      15        1   3.151880
3     -0.076507  0.142556      16        1   2.158091
15     0.257064 -0.061920      17        1   1.846937
10     0.127445 -0.163137      18        1   1.533599
12     0.281856  0.013628      19        1   1.198752
1      0.325190  0.033411      20        1   0.825782, topic_info=            Term          Freq         Total Category  logprob  loglift
696        woman  21977.000000  21977.000000  Default  30.0000  30.0000
493      percent  24425.000000  24425.000000  Default  29.0000  29.0000
2751       event   9095.000000   9095.000000  Default  28.0000  28.0000
1129      medium  15381.000000  15381.000000  Default  27.0000  27.0000
503       police  13571.000000  13571.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
3580  competitor    266.533581    750.644463  Topic20  -4.9361   3.7612
2752       fault    239.671536    486.167690  Topic20  -5.0424   4.0893
2059        gold    238.686170    756.985362  Topic20  -5.0465   3.6424
913     presence    245.086138   1531.507762  Topic20  -5.0200   2.9642
100     official    261.466392  11648.915341  Topic20  -4.9553   0.9999

[1304 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1879      3  0.055052    .
1879      5  0.274056    .
1879      6  0.564721    .
1879      7  0.028397    .
1879     10  0.007903    .
...     ...       ...  ...
704      10  0.034348   ’s
704      12  0.005434   ’s
704      15  0.004075   ’s
704      16  0.003493   ’s
704      19  0.008635   ’s

[3148 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 17, 14, 20, 15, 12, 6, 5, 1, 18, 7, 3, 8, 10, 19, 4, 16, 11, 13, 2])

In [ ]:
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.552204332026855

Coherence Score:  0.5094587503449799


In [ ]:
# Get topic weights and dominant topics ------------
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
import numpy as np  

# Get topic weights
topic_weights = []
for i, row_list in enumerate(ldamodel[doc_term_matrix]):
    topic_weights.append([w for i, w in row_list[0]])

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

TypeError: cannot unpack non-iterable int object